# NYT Comment Analysis
* Build csv file of comments from NYT articles:
    * Read raw bytes object from NYT API
    * Convert to json object(s)
    * Extract data to csv format

In [ ]:
# Modules
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
import json
import math
import numpy as np
import scipy as sp
from scipy import stats
from scipy.stats import pearsonr

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as col
import matplotlib.cm as cm
import seaborn as sns; sns.set()
from datetime import datetime

from collections import Counter
from bs4 import BeautifulSoup as soup
import requests as r
import urllib3
import zipfile
import re
import glob
import os
import time
from urllib.request import urlopen

# Pandas view options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
pd.set_option('precision', 4)

#Set PATH:
#PATH = ***

## Read Data

In [ ]:
# URL for data file

#Creating the columns and empty CSV files where data will be stored
comment_cols = (["commentID","status","commentSequence","userID","userDisplayName","userLocation","userTitle","userURL","picURL","commentTitle","commentBody","createDate","updateDate","approveDate","recommendations","replyCount","replies","editorsSelection","parentID","parentUserDisplayName","depth","commentType","trusted","recommendedFlag","permID","isAnonymous"])
comment_csv = []

#List of articles from which to pull comments
article_list = ["https://www.nytimes.com/2019/05/22/opinion/health-care-privacy-hipaa.html?rref=collection%2Fseriescollection%2Fnew-york-times-privacy-project","https://www.nytimes.com/2019/04/10/opinion/insurance-ai.html?rref=collection%2Fseriescollection%2Fnew-york-times-privacy-project"]

#Pulling comments from articles and saving in comment_cols
for i in range(len(article_list)):
    #defining the URL dependent on the URLs coming from the NYT articles themselves
    theURL = str("https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key=REDACTED&offset=0&url=" +article_list[i])
    pageSource = urlopen(theURL).read() #byte type
    s_page = pageSource.decode() #as string
    #num_comments = int(s_page[s_page.find("totalParentCommentsFound")+26:s_page.find("totalParentCommentsFound")+29])#will only work with a number in the 100s
    num_comments= int(s_page[s_page.find("totalCommentsFound")+20:s_page.find("totalCommentsFound")+23])#will only work with a number in the 100s
    time.sleep(6)
    for f in range(0,num_comments//25):
        print(len(comment_csv))
        theURLoff=str("https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key=REDACTED&offset="+str(f*25)+"&url=" +article_list[i])
        commSource=urlopen(theURLoff).read()
        c_page = commSource.decode()
        c_page = "["+c_page[208:c_page.find(',"depthLimit')] #cutting away the non-comment pieces
        data = json.loads(c_page)
        time.sleep(6)
        for comment in data:
            comment_temp=[]
            reply_temp=[]
            # Build comments csv
            for key in comment_cols:
                comment_temp.append(comment[key])
            comment_csv.append(comment_temp)
    
            #Add replies to comments csv if replies exist
            if comment['replyCount'] != 0:
            #for j in range(comment["replyCount"]):  #Trying to figure out how to get beyond the 1st reply
                for key in comment_cols:
                    reply_temp.append(comment['replies'][0][key])
                comment_csv.append(reply_temp)
                #if comment['replyCount'] >1:
                   # for key in comment_cols:
                   #     reply_temp.append(comment['replies'][1][key])
                   # comment_csv.append(comment_temp)
    
                   
comment_df = pd.DataFrame(comment_csv, columns=comment_cols)


In [ ]:
#Adding article_id column
#article_ids = np.zeros(len(comments_df))
#for i in range(200,len(article_ids)):
#    article_ids[i] = 1
    
    
#comment_df['articleIds'] = article_ids

In [ ]:
#Playground for troubleshooting the loop

# comment_cols = (["commentID","status","commentSequence","userID","userDisplayName","userLocation","userTitle","userURL","picURL","commentTitle","commentBody","createDate","updateDate","approveDate","recommendations","replyCount","replies","editorsSelection","parentID","parentUserDisplayName","depth","commentType","trusted","recommendedFlag","permID","isAnonymous"])
# comment_csv = []

# reply_cols = (["commentID","status","commentSequence","userID","userDisplayName","userLocation","userTitle","userURL","picURL","commentTitle","commentBody","createDate","updateDate","approveDate","recommendations","replyCount","replies","editorsSelection","parentID","parentUserDisplayName","depth","commentType","trusted","recommendedFlag","permID","isAnonymous"])
# reply_csv = []

# article_list = ["https://www.nytimes.com/2019/05/22/opinion/health-care-privacy-hipaa.html?rref=collection%2Fseriescollection%2Fnew-york-times-privacy-project","https://www.nytimes.com/2019/04/10/opinion/insurance-ai.html?rref=collection%2Fseriescollection%2Fnew-york-times-privacy-project"]

# #for i in range(len(article_list)):
#     #defining the URL dependent on the URLs coming from the NYT articles themselves
# theURL = str("https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key=REDACTED&offset=0&url=" +article_list[1])
# pageSource = urlopen(theURL).read() #byte type
# s_page = pageSource.decode() #as string
# num_comments = int(s_page[s_page.find("totalCommentsFound")+20:s_page.find("totalCommentsFound")+23]) #will only work with a number in the 100s
    

#     for f in range(0,num_comments//25):
#         theURLoff=str("https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key=REDACTED&offset="+str(f*25)+"&url=" +article_list[i])
#         commSource=urlopen(theURLoff).read()
#         c_page = commSource.decode()
#         c_page = "["+c_page[208:c_page.find(',"depthLimit')] #cutting away the non-comment pieces
#         data = json.loads(c_page)
        
#         for comment in data:
#             comment_temp=[]
#             reply_temp=[]
#             # Build comments csv
#             for key in comment_cols:
#                 comment_temp.append(comment[key])
#             comment_csv.append(comment_temp)
    
#             #Build reply csv if replys exist
#                if comment['replyCount'] != 0:
#                  for key in reply_cols:
#                     reply_temp.append(comment['replies'][0][key])
#                 reply_csv.append(reply_temp)
        
# comments_df = pd.DataFrame(comment_csv, columns=comment_cols)
# #reply_df = pd.DataFrame(reply_csv, columns=reply_cols)

# comments_df.head()








In [ ]:
# Save csv
#comments_df.to_csv(PATH + 'comments.csv')

## Analyze Data

In [ ]:
comments_df.head(20)

In [ ]:
comments_df.describe()